In [6]:
##arxiv--research
##tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

ModuleNotFoundError: No module named 'langchain_community'

In [83]:
##used inbuilt wikipedia tool
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [84]:
api_wrapper_arxiv = ArxivAPIWrapper(max_results=1,doc_content_chars_max=250)
Arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
Arxiv.name

'arxiv'

In [85]:
tools = [wiki,Arxiv]

In [86]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [87]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})

In [88]:
##custom tool creation[rag tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter




In [89]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DBF7422080>, search_kwargs={})

In [90]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [91]:
tools=[wiki,Arxiv,retriver_tool]

In [92]:
tools


[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\LANGCHAIN_PROJECTS\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='AI-search', description='useful for when you need to find information about artificial intelligence', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001DBADC92710>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object

In [93]:
##run agents with agents and llm models
##tools,LLM -->AGENTEXECUTOR


In [94]:
# import openai
# from langchain_openai import OpenAIEmbeddings

In [95]:
groq_api=os.getenv("GROQ_API_KEY")
# openai.api_key=os.getenv("OPENAI_API_KEY")
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant")

In [96]:
##prompt template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [97]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [98]:
##agent execution
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [99]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langsmith'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of languag
Invoking: `AI-search` with `{'query': 'Langsmith'}`


Jacque Fresco
Jakob von Uexküll
Jason Jixuan Hu
Jay Wright Forrester
Jennifer Wilby
John N. Warfield
Kevin Warwick
Ludwig von Bertalanffy
Maleyka Abbaszadeh
Manfred Clynes
Margaret Mead
Marian Mazur
N. Katherine Hayles
Natalia Bekhtereva
Niklas Luhmann
Norbert Wiener
Pyotr Grigorenko
Qian Xuesen
Ranulph Glanville
Robert Trappl
Sergei P. Kurdyumov
Anthony Stafford Beer
Stuart Kauffman
Stuart Umpleby
Talcott Parsons
Ulla Mitzdorf
Valentin Turchin
Valentin Braitenberg
William Ross Ashby
Walter Bradford Cannon
Walter Pitts
Warren McCulloch
William Grey Walter

GPT

भाषा日本語NordfriiskNorsk bokmålNor

{'input': 'Tell me about Langsmith',
 'output': "It appears that Langsmith is not a widely recognized term in the field of artificial intelligence or computer science. However, based on the information provided, it's possible that Langsmith is a proprietary or custom-built language model integration framework, similar to LangChain. If you could provide more context or information about Langsmith, I may be able to provide a more accurate response."}

In [101]:
agent_executor.invoke({"input":"what is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `AI-search` with `{'query': 'machine learning'}`


Learning
Machine learning is the study of programs that can improve their performance on a given task automatically.[41] It has been a part of AI from the beginning.[e]

Expectation–maximization clustering of Old Faithful eruption data starts from a random guess but then successfully converges on an accurate clustering of the two physically distinct modes of eruption.
Classifiers and statistical learning methods
The simplest AI applications can be divided into two types: classifiers (e.g., "if shiny then diamond"), on one hand, and controllers (e.g., "if diamond then pick up"), on the other hand. Classifiers[99] are functions that use pattern matching to determine the closest match. They can be fine-tuned based on chosen examples using supervised learning. Each pattern (also called an "observation") is labeled with a certain predefined class. All the observations combined with their cl

{'input': 'what is machine learning',
 'output': 'Unfortunately, I was unable to find enough information with the available functions to answer your question.'}

In [104]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about in brief?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': '1706.03762'}`
responded: It seems that the paper '1706.03762' does not exist in arXiv. Let's try to find information about it on Wikipedia or through AI-search.



Page: Generative pre-trained transformer
Summary: A generative pre-trained transformer (GPT) is a type of large language model (LLM) that is widely used in generative AI chatbots. GPTs are based on a deep learning architecture called the transformer.The paper "Attention Is All You Need" with the paper ID 1706.03762 is about a generative pre-trained transformer architecture, specifically the transformer. It's a type of large language model (LLM) that'

{'input': "What's the paper 1706.03762 about in brief?",
 'output': 'The paper "Attention Is All You Need" with the paper ID 1706.03762 is about a generative pre-trained transformer architecture, specifically the transformer. It\'s a type of large language model (LLM) that\'s widely used in generative AI chatbots.'}